In [25]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Packed_items.csv')

In [3]:
df.head()

,Order_ID,Layer_Index,Product_ID,x,y,w,h,Quantity
0,1,0,2,0,0,95,58,10
1,1,0,5,0,58,116,38,8
2,1,0,0,95,0,19,58,2
3,1,1,4,0,0,58,57,6
4,1,1,12,0,57,80,26,2


In [37]:
failed_checks = []


for order_id, order in df.groupby('Order_ID'):
    for layer_id, layer in order.groupby('Layer_Index'):
        #iterate over all orders and layers

        product_ids = layer['Product_ID'].values
        i, c = np.unique(product_ids, return_counts=True)
        ids_to_check = i[c > 1]
        if ids_to_check.size == 0:
            continue

        for item_id in ids_to_check:
            
            connection_graph = nx.Graph()
            items_to_check = layer[layer['Product_ID'] == item_id]

            for i, item in items_to_check.iterrows():

                connection_graph.add_node(i)
                for node in connection_graph.nodes:
                    other_item = items_to_check.loc[node]

                    e = 1

                    x_overlap = max(item.x-e,other_item.x)
                    y_overlap = max(item.y-e,other_item.y)
                    w_overlap = min(item.x+item.w+e,other_item.x+other_item.w) - x_overlap
                    h_overlap = min(item.y+item.h+e,other_item.y+other_item.h) - y_overlap

                    if w_overlap > 0 and h_overlap > 0:
                        connection_graph.add_edge(i,node)



            if not nx.is_connected(connection_graph):
                failed_checks.append((order_id, layer_id, item_id))


failed_checks_df = pd.DataFrame(failed_checks, columns=['Order_ID', 'Layer_Index', 'Product_ID'])
print(len(failed_checks_df))
print(failed_checks_df['Order_ID'])


        



715
0         5
1         8
2        15
3        16
4        22
       ... 
710    2484
711    2485
712    2493
713    2494
714    2496
Name: Order_ID, Length: 715, dtype: int64
